# Montevideo: Model

We build a model for finetuning a ResNet-50 convnet pretrained with ImageNet.
Basically, we load the ResNet-50 architecture without the top layer, and add on top of it 4 additional layers:
* Flatten
* Dense (1024 outputs)
* Dropout (0.5)
* Dense (3 outputs)

The last layer refers to the 3 output classes, and its activation layer is a Sigmoid (because labels are not mutually exclusive).
The training schedule consists of 2 steps:
1. Train last 4 layers, to account for randomized weights in the new 4 layers
2. Train last 30 layers

In [1]:
#@title Check GPU presence { display-mode: "form" }

import tensorflow as tf
print("GPU device name: {}".format(tf.test.gpu_device_name()))

GPU device name: /device:GPU:0


In [2]:
#@title Download datasets { display-mode: 'form' }
#@test {'output': 'ignore'}

#!mkdir -p 1/
#!gcloud config set project golden-system-178513
#!gsutil -m cp gs://dym-temp/school-mapping/datasets/1.tar.gz 1/
#!cd 1/ && tar xzf 1.tar.gz

In [3]:
import numpy as np
import os
import csv
import gc

from glob import glob
from keras.applications.resnet50 import ResNet50
from keras.layers import Flatten, Dense, Dropout
from keras.models import Model
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras import optimizers
from keras import backend as K 

Using TensorFlow backend.


In [4]:
DATASET_DIR = '../data/ds2/'

WIDTH = 300
HEIGHT = 300
CLASSES = ('urban', 'rural', 'school')

In [5]:
TRAIN_DIR = os.path.join(DATASET_DIR, 'train')
VAL_DIR = os.path.join(DATASET_DIR, 'test')

train_files = glob(os.path.join(TRAIN_DIR, '*.jpg'))
val_files = glob(os.path.join(VAL_DIR, '*.jpg'))

n_train_samples = len(train_files)
n_val_samples = len(val_files)

n_train_samples, n_val_samples

(6055, 9054)

In [44]:
model = ResNet50(weights='imagenet',
                 include_top=False,
                 input_shape=(WIDTH, HEIGHT, 3))

/home/ubuntu/.local/lib/python3.5/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [6]:
FC_SIZE = 1024
DROPOUT = 0.5

In [46]:
x = model.output
x = Flatten()(x)
x = Dense(FC_SIZE, activation='relu')(x)
x = Dropout(DROPOUT)(x)
predictions = Dense(len(CLASSES), activat
                    
                    ion='sigmoid')(x)

In [47]:
LR = 0.0001
#MOMENTUM = 0.9

In [48]:
model_final = Model(inputs=model.input, outputs=predictions)

model_final.compile(loss='binary_crossentropy',
                    optimizer=optimizers.RMSprop(lr=LR),
                    metrics=['accuracy'])

In [49]:
model_final.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 300, 300, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 306, 306, 3)  0           input_4[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 150, 150, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 150, 150, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

## Data augmentation

We augment the dataset with multiple transformations, like flipping, rotatoins and brightness randomization.

In [7]:
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator

In [8]:
BATCH_SIZE = 40

In [9]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=(1.0, 1.0),
    fill_mode="nearest",
    zoom_range=0.3,
    width_shift_range=0.3,
    height_shift_range=0.3,
    rotation_range=30)

test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input)

In [10]:
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    classes=[''],
    target_size=(HEIGHT, WIDTH),
    batch_size=BATCH_SIZE,
    class_mode="categorical")

validation_generator = test_datagen.flow_from_directory(
    VAL_DIR,
    classes=[''],    
    target_size=(HEIGHT, WIDTH),
    class_mode="categorical")

Found 6055 images belonging to 1 classes.
Found 9054 images belonging to 1 classes.


In [11]:
def parse_label_row(row):
    return [int(row[k]) for k in CLASSES]

def read_labels_dict(dataset_dir):
    with open(os.path.join(dataset_dir, 'labels.csv')) as csvfile:
        reader = csv.DictReader(csvfile)
        return {row['img']: parse_label_row(row) for row in reader}

In [12]:
labels_dict = read_labels_dict(DATASET_DIR)

In [13]:
def build_data_generator(gen, labels_dict):
    for x in gen:
        idx = gen.batch_index * gen.batch_size
        filenames = gen.filenames[idx : idx + gen.batch_size]
        labels = np.array([labels_dict[fname] for fname in filenames])
        
        if x[0].shape[0] == labels.shape[0]:
            yield x[0], labels

In [14]:
train_datagen = build_data_generator(train_generator, labels_dict)
val_datagen = build_data_generator(validation_generator, labels_dict)

## Training

In [15]:
CLASS_WEIGHT = {0: 0.46620047, 1: 1.21281031, 2: 32.82051282}

In [16]:
WEIGHTS_PATH = "weights-3.h5"

In [17]:
from keras.callbacks import Callback
import subprocess

class UploadToStorageCallback(Callback):
    def __init__(self, gspath):
        super(UploadToStorageCallback, self).__init__()
        self.gspath = gspath
        
    def on_epoch_end(self, epoch, logs=None):
        cmd = 'gsutil -m cp -n *.h5 {}'.format(self.gspath)
        print(cmd)
        subprocess.run(cmd, shell=True)

In [18]:
checkpoint = ModelCheckpoint(WEIGHTS_PATH,
    monitor = 'val_acc',
    verbose = 1,
    save_best_only = True,
    save_weights_only = False,
    mode = 'auto',
    period = 1)

upload_to_storage = UploadToStorageCallback(
    'gs://dym-temp/school-mapping/models/')

early = EarlyStopping(
    monitor = 'val_acc',
    min_delta = 0,
    patience = 10,
    verbose = 1,
    mode = 'auto')

reduce_lr = ReduceLROnPlateau(
    monitor = 'val_loss',
    factor = 0.2,
    patience = 5,
    min_lr = 0.0001)

In [19]:
SCHEDULE = [
    dict(epochs=5, lr=0.001, layers=4),
    dict(epochs=30, lr=0.001, layers=30),
]

In [20]:
def freeze_last_layers(model, n_layers):
    for layer in model.layers:
        layer.trainable = False
    for layer in model.layers[-n_layers:]:
        layer.trainable = True
        
def build_model():
    model = ResNet50(weights='imagenet',
                     include_top=False,
                     input_shape=(WIDTH, HEIGHT, 3))

    x = model.output
    x = Flatten()(x)
    x = Dense(FC_SIZE, activation='relu')(x)
    x = Dropout(DROPOUT)(x)
    predictions = Dense(len(CLASSES), activation='sigmoid')(x)

    return Model(inputs=model.input, outputs=predictions)

In [ ]:
model = build_model()

histories = []
for i, opts in enumerate(SCHEDULE):
    print("===== Schedule step {} =====".format(i))

    freeze_last_layers(model, opts['layers'])

    model.compile(loss='binary_crossentropy',
        optimizer=optimizers.SGD(lr=opts['lr']),
        metrics=['acc'])
    
    history = model_final.fit_generator(
        train_datagen,
        steps_per_epoch = n_train_samples // BATCH_SIZE,
        epochs = opts['epochs'], 
        validation_data = val_datagen,
        validation_steps = n_val_samples // BATCH_SIZE,
        #class_weight = CLASS_WEIGHT,
        #callbacks = [checkpoint, early, reduce_lr])
        #callbacks = [checkpoint, early, upload_to_storage])
        callbacks = [checkpoint, early])
      
    histories.append(history)
    model.load_weights(WEIGHTS_PATH)

In [ ]:
#!gsutil -m cp -n *.h5 gs://dym-temp/school-mapping/models/

In [79]:
K.clear_session()
del model
gc.collect()

47113